<a href="https://colab.research.google.com/github/rifatislam-25/Amazon-Sales-analysis-with-jupyter/blob/main/Cv_Assistant_Chatbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -U langchain-community
!pip install groq
!pip install faiss-cpu
!pip install langchain-groq
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS  # ✅ Updated Import
from langchain_community.embeddings import HuggingFaceEmbeddings  # ✅ Updated Import
from langchain.chat_models import ChatOpenAI
from groq import Groq
from sentence_transformers import SentenceTransformer
import torch
from langchain_groq import ChatGroq

# Secure API Key Handling (Set this in your environment)
os.environ["GROQ_API_KEY"] = "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"

# ✅ Load Pretrained Embedding Model
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# ✅ CV Text Data
cv_text = """
Name: Md Shafiqul Islam
Contact: [Your Email] | [Your Phone Number] | [LinkedIn Profile]

🎓 **Education**:
- MSc in Electrical and Electronic Engineering, North South University (Major: Network and Telecommunication)
- BSc in Electrical and Electronic Engineering, North South University (Major: Power)

🛠️ **Technical Skills**:
- **Programming**: Python (Pandas, NumPy, Matplotlib), SQL, Power BI
- **Data Analysis**: Time Series Forecasting, Data Visualization, Machine Learning Basics
- **AI & Automation**: Chatbot Development (LangChain, OpenAI API), AI Agents, RAG-based AI
- **Power & Renewable Energy**: Floating Solar Power Plant, Microstrip Patch Antenna Design
- **Tools**: Jupyter Notebook, Google Colab, n8n, Make.com, Power Automate

💼 **Work Experience**:
- **Assistant Project Manager | Floating Solar Power Plant (Bangladesh)**
  - Led project operations and managed logistics for Bangladesh’s first **floating solar power plant**.
  - Coordinated between engineers, vendors, and government officials.
  - Analyzed project efficiency and optimized renewable energy solutions.

📊 **Projects**:
- **AI-Powered CV Assistant**
  - Developed a chatbot using LangChain + OpenAI to answer CV-related questions.
- **Weather Forecasting Dashboard**
  - Built a Power BI dashboard with real-time weather data using APIs and Facebook Prophet.
- **Credit Card Transaction Analysis**
  - Performed fraud detection using Power BI and SQL.

🏆 **Achievements & Certifications**:
- IELTS Score: **6.0**
- IBM Python for Data Science (Coursera) – In Progress

🌍 **Hobbies & Interests**: Reading, Traveling, AI & Automation, Renewable Energy Research
"""
extracurricular_activities = """
🎯 **Extracurricular Activities**:
- **Vice President** (Head of Logistics and Operations), North South University Social Services Club (March 2017 – March 2018)
  - Led and organized various events such as Socio Camp, a social idea-generating competition.
  - Managed logistics and operations for club events, ensuring smooth execution.
- **Sub Executive**, North South University Social Services Club (2016 – 2017)
  - Assisted in organizing social service programs and community outreach initiatives.
- **Active Member**, IEEE North South University Chapter (2016 – Present)
  - Participated in workshops, seminars, and technical events related to electrical engineering.
- **Blood Donation Drive Organizer**, North South University Social Services Club (2015 – 2018)
  - Coordinated blood donation drives to support local hospitals and the community.
"""

full_cv = cv_text + extracurricular_activities


# ✅ Step 1: Text Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Increased chunk size for better context
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", ", "]
)
cv_chunks = text_splitter.split_text(cv_text)

# ✅ Step 2: Convert Chunks into Vector Embeddings
cv_embeddings = embedding_model.encode(cv_chunks, convert_to_tensor=True)

# ✅ Step 3: Create a FAISS Vector Store for Fast Search
vector_store = FAISS.from_texts(cv_chunks, HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"))

# ✅ Step 4: Initialize Groq API (as OpenAI-compatible model)
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Step 4: Initialize Groq API
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"), #Ensure this is your Groq key
    model="llama-3.1-8b-instant",       # Or another Groq model
    temperature=0.3
)


# ✅ Step 5: Setup RAG-based Question Answering Pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True
)

# ✅ Step 6: Interactive Chatbot
def cv_assistant():
    print("\n📝 CV Assistant is ready! Ask me about your CV (type 'exit' to quit).")

    while True:
        question = input("\n❓ Your Question: ")
        if question.lower() == "exit":
            print("\n👋 Goodbye!")
            break

        response = qa_chain.invoke({"query": question})
        answer = response["result"]
        print(f"\n💬 Answer: {answer}")

# ✅ Start the Assistant
if __name__ == "__main__":
    cv_assistant()



📝 CV Assistant is ready! Ask me about your CV (type 'exit' to quit).

❓ Your Question: exit

👋 Goodbye!
